In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo
import pickle
from sklearn.preprocessing import RobustScaler,PowerTransformer
from keras.models import Sequential
from keras.layers import LSTM, Dense

d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\dask\dataframe\__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'      

scalers = {}
# Function to center, scale, and return a series
def scale_group(group):
    scaler = RobustScaler()
    #scaler = PowerTransformer()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Store the scaler for this group
    return pd.Series(scaled_values, index=group.index, name=group.name)

# Function to inverse transform (de-scale) and decenter, and return a series
def inverse_scale_group(group):
    group_name = group.name
    scaler = scalers[group_name]
    inversed_centered_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    original_values = inversed_centered_values
    return pd.Series(original_values, index=group.index, name=group_name)


In [3]:

# Leer datos
#df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
#import dask.dataframe as dd
df_final = pd.read_parquet(DATOS_DIR+'FE_07_dataset.parquet') 
#df_final = df_final.compute()

df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)




In [4]:

## ESTO ES UN HACK PORQUE ESTA MAL ARMADA LA FUNCIÓN DE ESCALADO Y SOLO GUARDA LOS DATOS DE LA ÜLTIMA APLICACIÓN

with open(DATOS_DIR + '/scalers.pkl', 'rb') as file:
    scalers = pickle.load(file)

#df_test = df_final[(df_final['product_id'] == 21276) & (df_final['customer_id'] == 10029)][['product_id','tn', 'weight']]
#df_test['tn'] = df_test.groupby('product_id')['tn'].transform(inverse_scale_group)

#df_test

In [5]:

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-10-01']
df_predict = df_final.loc['2018-01-01':'2019-10-01']

In [6]:
df_true = df_true[['product_id','customer_id','tn']]
df_true

,product_id,customer_id,tn
periodo,,,
2019-12,20307,10417,0.000000
2019-12,20320,10566,0.000000
2019-12,20600,10423,0.000000
2019-12,20322,10318,0.000000
2019-12,20296,10027,0.076756
...,...,...,...
2019-12,20203,10026,0.000000
2019-12,20203,10027,0.876033
2019-12,20680,10348,0.000000


In [7]:
df_true['tn'] = df_true.groupby(['product_id','customer_id'])['tn'].transform(inverse_scale_group)
pred_true = df_true.groupby(['product_id'])['tn'].sum()
pred_true.to_csv(DATOS_DIR+'/pred/TRUE-NOVEMBER.csv', index=True,header=True)


In [8]:
#df_final[df_final['product_id'] == 20402 & df_final['customer_id'] == 10479][['product_id','tn']]

In [9]:

#Filtro test
#df_final = df_final[df_final['product_id'] < 20013]
#df_final = df_final[df_final['product_id'] == 20072]

weight= df_final[['weight','product_id','customer_id']]
#df_final.drop(columns=['weight'], inplace=True)
#filtered_weight = weight[weight['weight'] != 0]
#weight_mean = filtered_weight.groupby('product_id')['weight'].mean().to_dict()
#weight['weight'] = weight['product_id'].map(weight_mean)

weight = weight.groupby(['product_id','customer_id'])['weight'].mean()
weight = weight.reset_index()
df_final = df_final.drop(columns=['weight'])
df_predict = df_predict.drop(columns=['weight'])

In [10]:
# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'Regression',
        'metric':'rmse',
        'verbose': -1,
        #'n_jobs': -1,
        'seed': 10000079,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}

predictions_all = pd.DataFrame(columns=['tn'])
products = df_final['product_id'].unique()
tot = len(products)
nro = 0
for producto in products:
    print(f'Fitting and predicting for product_id: {producto}')
    # Filtrar los datos del producto
    df_producto = df_final[df_final['product_id'] == producto]
    
    product_weights = pd.merge(df_producto.reset_index(), weight, on=['product_id', 'customer_id'], how='left')
    product_weights.set_index('periodo', inplace=True)
    weight_p = product_weights['weight']
    # Prepare data for LSTM on tn_2 only
    X = df_producto[['diff_tn_tn2']].values.astype('float32')
    y = df_producto['tn_2'].values.astype('float32')
    X = X.reshape((X.shape[0], 1, X.shape[1]))
    #######################################################    
    # Define LSTM model
    model = Sequential()
    model.add(LSTM(50, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # Train LSTM model
    model.fit(X, y, epochs=10, batch_size=32, verbose=1)

    # Extract features from LSTM
    features = model.predict(X)

    # Prepare data for LightGBM
    # Convert LSTM features to DataFrame
    features_df = pd.DataFrame(features, index=df_producto.index, columns=[f'lstm_feature_{i}' for i in range(features.shape[1])])
    df_producto = pd.concat([df_producto, features_df], axis=1)
    df_producto['tn_2'] = y
    #############################################################################  
    #### Agrupar y escalar
   
    model, average_metric = labo.train_lightgbm_model(df_producto,params,metric='rmse',
                                                      weights=weight_p)
    print("Overall rmse metric: ", average_metric)
    # Predict values for the entire dataset using the trained models
    # Prepare last data points for prediction
    df_predict_p = df_predict[df_predict['product_id'] == producto]
    last_data_points = df_predict_p[df_predict_p.index == df_predict_p.index.max()].copy()
    last_data_points.drop(columns=['tn_2'], inplace=True)
    # Predict the next month's value using the trained model
    predictions = labo.predict_next_month_customer(model, last_data_points)
    preds = predictions.groupby(['product_id','customer_id'])['tn_2'].transform(inverse_scale_group)
    predictions['tn'] = preds
    predictions.drop(columns=['tn_2'], inplace=True)
    predictions = predictions.reset_index()
    predictions =  predictions.groupby('product_id')['tn'].sum()
    predictions.columns = ['product_id', 'tn']
    predictions_all = pd.concat([predictions_all, predictions])
    print(predictions_all[-1:])


Fitting and predicting for product_id: 20818
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[19]	validation's rmse: 0.913588
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[12]	validation's rmse: 0.553838
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[50]	validation's rmse: 2.32638
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[458]	validation's rmse: 2.28356
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[146]	validation's rmse: 3.32503
Overall rmse metric:  0.30673651821722675
              tn
20818  10.068288
Fitting and predicting for product_id: 20200
Training until validation scores don't improve for 10 rounds


C:\Users\Admin\AppData\Local\Temp\ipykernel_28636\90720821.py:52: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])
C:\Users\Admin\AppData\Local\Temp\ipykernel_28636\90720821.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_all = pd.concat([predictions_all, predictions])


Early stopping, best iteration is:
[37]	validation's rmse: 0.635502
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[44]	validation's rmse: 0.506074
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[87]	validation's rmse: 0.481474
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[41]	validation's rmse: 0.425639
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[47]	validation's rmse: 0.317167
Overall rmse metric:  0.10059466366749674
              tn
20200  23.525179
Fitting and predicting for product_id: 20292
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[68]	validation's rmse: 0.38044
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[120]	validation's rmse: 0.530474
Training until validation scores don'

In [11]:

predictions_all['tn']=predictions_all['tn'].astype('float32')
predictions_all.index.names = ['product_id']
predictions_all.to_csv(DATOS_DIR+'/pred/0030-prediccion-RSME_scaled_Robust-product_id-Outliers.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


Overall custom metric:  0.13165577519340693
